# 1. 讀取檔案

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# 使用 pandas 的 read_csv 函式讀取訓練資料
df = pd.read_csv("data/train.csv")

# 2.資料預處理

In [26]:
# 直接刪除任何含有缺失值的整行資料
# HINT：有更好的預處理填補方式嗎？
df_clean = df.fillna(df.median(numeric_only=True))

# 3.特徵工程

In [27]:
# 定義要用來預測的特徵欄位
# HINT：這串列遺漏了部分特徵，另外，有方法額外加上新的特徵嗎？
features = ['weight', 'acceleration', 'model_year', 'cylinders', 'displacement', 'horsepower']
df_clean['weight_per_hp'] = df_clean['weight'] / df_clean['horsepower']
df_clean['displacement_per_cyl'] = df_clean['displacement'] / df_clean['cylinders']
df_clean['age'] = 2025 - df_clean['model_year']  
features += ['weight_per_hp', 'displacement_per_cyl', 'age']

# 定義我們要預測的目標欄位
target = 'mpg'

# 從乾淨的資料中選取 X 和 y
X = df_clean[features]
y = df_clean[target]


# 4. 訓練模型

In [28]:
# 分割訓練集與測試集 (用於本地評估模型好壞)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#特徵標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 建立並訓練模型
model = LinearRegression()
model.fit(X_train, y_train)

# 進行預測與評估
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print("模型已訓練完成！")
print(f"訓練誤差 (Train RMSE): {train_rmse:.4f} MPG")
print(f"測試誤差 (Test RMSE):  {test_rmse:.4f} MPG")
print("\n--- 模型學到的關係 ---")

# 各個特徵的權重
for feature, coef in zip(features, model.coef_):
    print(f"特徵 '{feature}' 的權重: {coef:.4f}")

模型已訓練完成！
訓練誤差 (Train RMSE): 2.9970 MPG
測試誤差 (Test RMSE):  3.4200 MPG

--- 模型學到的關係 ---
特徵 'weight' 的權重: -0.0062
特徵 'acceleration' 的權重: -0.2290
特徵 'model_year' 的權重: 0.4311
特徵 'cylinders' 的權重: -4.9096
特徵 'displacement' 的權重: 0.1356
特徵 'horsepower' 的權重: 0.0192
特徵 'weight_per_hp' 的權重: 0.3086
特徵 'displacement_per_cyl' 的權重: -0.8921
特徵 'age' 的權重: -0.4311


# 5.輸出提交檔案

In [29]:
# 讀取需要進行預測的測試檔案 test.csv
df_test = pd.read_csv("data/test.csv")

# 對測試資料進行預處理
# HINT：如果前面使用了其他的預處理方式，這邊要如何修改？
df_test = df_test.fillna(df_clean.median(numeric_only=True))
df_test['weight_per_hp'] = df_test['weight'] / df_test['horsepower']
df_test['displacement_per_cyl'] = df_test['displacement'] / df_test['cylinders']
df_test['age'] = 2025 - df_test['model_year']  

# 使用訓練好的模型，對測試資料進行預測
predictions = model.predict(df_test[features])

# 建立一個新的 DataFrame
submission_df = pd.DataFrame({'Id':df_test['id'], 'mpg': predictions})

# 保存為 submission.csv
submission_df.to_csv('submission.csv', index=False)
print("提交文件 'submission.csv' 已成功生成！")

提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：_劉嘉芸___ 學號：___111108506____

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：__2.67191__  

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。  
【實驗 1】  
    我做的修改：_____將刪除缺失值改成用中位數填補、加入更多特徵：weight_per_hp（重量除以馬力）,displacement_per_cyl（排氣量除以汽缸數）______  
    結果與觀察 (分數變化、心得等)：___如果直接刪除所有缺失值會遺失太多資料，改成中位數之後有所改善、多加一點特徵，分數馬上就降低了，更多特徵的準確度就越好____  
    該次實驗分數： _______2.67191_____  
【實驗 2】  
    我做的修改：__加入超多其他特徵，然後進行標準化處理__  
    結果與觀察 (分數變化、心得等)：___結果分數沒有比較好，應該是因為新加入的特徵跟原本的有線性相關__  
    該次實驗分數： _2.82220_  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：__在實驗的時候發現特徵工程跟資料前處理的方式最影響準確度，將刪除資料的方法改為用中位數填補分數就變好了但特徵就不是越多越好，如果加入比較不相關的特徵或是有線性關係的特徵，反而準確度會降低。其實做了這麼多次修改，反而一開始的結果是最好的，可見特徵不需要亂加，夠用就好，基本上影響最大的應該是資料前處理。___    